# Введение в нейронные сети
## Урок 8. GAN

### Обучите нейронную сеть любой архитектуры, которой не было на курсе, либо нейронную сеть разобранной архитектуры, но на том датасете, которого не было на уроках. Сделайте анализ того, что вам помогло в улучшения работы нейронной сети

In [1]:
# !pip install opendatasets

In [2]:
import opendatasets as od

In [3]:
od.download("https://github.com/JurgenPalsma/swarmex/archive/refs/heads/master.zip")

20684800it [00:02, 10296898.32it/s]


In [4]:
!unzip "/content/swarmex-master.zip" -d "/content/"

Archive:  /content/swarmex-master.zip
00453379dfe6fef3416dcac85a80c958c01bac07
   creating: /content/swarmex-master/
  inflating: /content/swarmex-master/.gitignore  
  inflating: /content/swarmex-master/README.md  
   creating: /content/swarmex-master/analysis/
  inflating: /content/swarmex-master/analysis/indepth_csfla_tuning.html  
  inflating: /content/swarmex-master/analysis/indepth_csfla_tuning.ipynb  
  inflating: /content/swarmex-master/analysis/indepth_pso_tuning.html  
  inflating: /content/swarmex-master/analysis/indepth_pso_tuning.ipynb  
  inflating: /content/swarmex-master/analysis/preliminary_csfla_tuning_analysis.html  
  inflating: /content/swarmex-master/analysis/preliminary_csfla_tuning_analysis.ipynb  
  inflating: /content/swarmex-master/analysis/preliminary_pso_tuning_analysis.html  
  inflating: /content/swarmex-master/analysis/preliminary_pso_tuning_analysis.ipynb  
  inflating: /content/swarmex-master/analysis/test_data_analysis.html  
  inflating: /content/swa

In [ ]:
%cd swarmex-master/

/content/swarmex-master


In [5]:
# !pip install py4j

     |████████████████████████████████| 198 kB 5.4 MB/s 


In [ ]:
!python main.py -c ./config/demo.json

Transaction cost: 1.0E-5
Slippage allowance: 0.0
Loading directional changes data...
DC curves:
0.010%: {length =   619, meanRatio[up] = 0.68, meanRatio[down] = 0.48}
0.013%: {length =   569, meanRatio[up] = 0.71, meanRatio[down] = 0.53}
0.015%: {length =   503, meanRatio[up] = 0.73, meanRatio[down] = 0.61}
0.018%: {length =   473, meanRatio[up] = 0.76, meanRatio[down] = 0.63}
0.020%: {length =   449, meanRatio[up] = 0.73, meanRatio[down] = 0.67}

Starting GA...
Random seed: 1643320951368
Training budget: 500000.0
Test budget: 500000.0

Generation	Best	Worst	Average
0	    0.000674	-9999.000000	 -149.985049
1	    0.000723	-9999.000000	  -69.993015
2	    0.000839	-9999.000000	  -29.996948
3	    0.000847	   -0.000847	    0.000167
4	    0.000865	-9999.000000	   -9.998683
5	    0.000923	-9999.000000	  -19.997536
6	    0.000932	   -0.000529	    0.000610
7	    0.000947	-9999.000000	   -9.998320
8	    0.000947	   -0.000868	    0.000733
9	    0.000969	   -0.000429	    0.000762
10	    0.000992	-

Поиск для решения задачи начал со [статьи](https://iopscience.iop.org/article/10.1088/1742-6596/1641/1/012067/pdf). В статье выглядят перспективными результаты исследований:  
>"William H Hidayatno A and Zahra A A, 2014 Aplikasi Jaringan Saraf Tiruan Perambatan Balik Untuk
Prakiraan Valuta GBP/USD Dalam Forex Trading Transient 3, 4 p. 1–8."  

В связи с этим решил использовать нейросеть с использованием _partical swarm optimization_.  
Для применения модели выбрал работу JurgenPalsma на [github](https://github.com/JurgenPalsma/swarmex).  
Полноценный тестовый запус осуществить не удалось, поэтому обратился к другому источнику.

In [6]:
# !pip install pyswarms

     |████████████████████████████████| 104 kB 5.4 MB/s 


In [ ]:
%cd ..
!pwd

/content
/content


In [7]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [8]:
# список признаков, по которым будем учить
channelIndexes = [0, 1, 2, 3]
#channelIndexes = [0,1,2]

# длина истории для работы
xLen = 14
# отступ тестов от тренировок
bias = 7

# шаг по данным для построения обучающих примеров
step = 1

# горизонт предсказания
future = 1


# параметры модели для обучения
batch_size = 8
epochs = 10

train_split_percents = 80

In [9]:
def data_get(path_to_csv, start_date, batch_size):
  df = pd.read_csv(path_to_csv, usecols=['<DATE>','<HIGH>','<LOW>'],
                   index_col=['<DATE>'], parse_dates=['<DATE>'],) #sep=';')
  df = df[df.index >= pd.to_datetime(start_date)]
  df["HLAvg"] = df['<HIGH>'].add(df['<LOW>']).div(2)
  del df['<HIGH>']
  del df['<LOW>']

  # Simple Moving Average
  df['MA'] = df['HLAvg'].rolling(window=14).mean()
  # Log Returns
  df['Returns'] = np.log(df['MA']/df['MA'].shift(1))
  
  df.dropna(how='any', inplace=True)

  return df

#### Попытка применения нейросети на данных с котировками CHFUSD

In [10]:
df = data_get('./CHFUSD_210101_220118_1H.csv', '01.01.2021', batch_size)
# Simple Moving Average
df['MA'] = df['HLAvg'].rolling(window=xLen).mean()
# Log Returns
df['Returns'] = np.log(df['MA']/df['MA'].shift(1))
df.dropna(how='any', inplace=True)
df = df[df.shape[0] % batch_size:]
validation_size = round(df.shape[0]*0.2)
test_size = round(df.shape[0]*0.2)
df_train = df[:- validation_size - test_size]
df_validation = df[- validation_size - test_size - xLen:- test_size]
df_test = df[- test_size - xLen:]
df_train.shape, df_validation.shape, df_test.shape

2022-01-30 23:31:37,608 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


((4066, 3), (1369, 3), (1369, 3))

In [11]:
def get_train(values, window_size):
    X, y = [], []
    len_values = len(values)
    for i in range(window_size, len_values):
        X.append(values[i-window_size:i])
        y.append(values[i])
    X, y = np.asarray(X), np.asarray(y)
    X = np.reshape(X, (X.shape[0], X.shape[1]))
    y = np.reshape(y, y.shape[0])
    return X, y

def get_val(values, window_size):
    X = []
    len_values = len(values)
    for i in range(window_size, len_values):
        X.append(values[i-window_size:i])
    X = np.asarray(X)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    y = values[-X.shape[0]:]
    return X, y

In [13]:
X, y = get_train(df[['Returns']].values, xLen)
y[y<-0.000001] = -1
y[y>0.000001] = 1
y = np.where((y>-0.000001)&(y<0.000001), 0, y)
y = y.astype(int)
X.shape, y.shape

((6762, 14), (6762,))

In [14]:
# Forward propagation
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss. It receives a set of parameters that must be
    rolled-back into the corresponding weights and biases.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """
    # Neural network architecture
    n_inputs = 14
    n_hidden = 370
    n_classes = 3

    # Roll-back the weights and biases
    W1 = params[0:5180].reshape((n_inputs,n_hidden))
    b1 = params[5180:5550].reshape((n_hidden,))
    W2 = params[5550:6660].reshape((n_hidden,n_classes))
    b2 = params[6660:6663].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood
    N = 6762 # Number of samples
    corect_logprobs = -np.log(probs[range(N)])
    loss = np.sum(corect_logprobs) / N

    return loss

In [15]:
def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [17]:
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = y.shape[0]
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=100, verbose=3)

2022-01-30 23:33:11,132 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=3.3
2022-01-30 23:53:59,264 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 3.2960114346644582, best pos: [0.28784608 0.54720764 0.02994148 ... 0.77016003 0.50483204 0.11480229]


In [21]:
def predict(X, pos):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    X: numpy.ndarray
        Input Iris dataset
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    # Neural network architecture
    n_inputs = 14
    n_hidden = 370
    n_classes = 3

    # Roll-back the weights and biases
    W1 = pos[0:5180].reshape((n_inputs,n_hidden))
    b1 = pos[5180:5550].reshape((n_hidden,))
    W2 = pos[5550:6660].reshape((n_hidden,n_classes))
    b2 = pos[6660:6663].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [22]:
(predict(X, pos) == y).mean()

0.0048802129547471165

#### Попытка применения нейросети по [образцу](https://pyswarms.readthedocs.io/en/development/examples/custom_objective_function.html)

In [ ]:
# Load the iris dataset
data = load_iris()

# Store the features as X and the labels as y
X = data.data
y = data.target

In [ ]:
X.shape, y.shape

((150, 4), (150,))

In [ ]:
X[0].shape, X[160:163]

((4,), array([], shape=(0, 4), dtype=float64))

In [ ]:
# Forward propagation
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss. It receives a set of parameters that must be
    rolled-back into the corresponding weights and biases.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """
    # Neural network architecture
    n_inputs = 4
    n_hidden = 20
    n_classes = 3

    # Roll-back the weights and biases
    W1 = params[0:80].reshape((n_inputs,n_hidden))
    b1 = params[80:100].reshape((n_hidden,))
    W2 = params[100:160].reshape((n_hidden,n_classes))
    b2 = params[160:163].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood
    N = 150 # Number of samples
    corect_logprobs = -np.log(probs[range(N), y])
    loss = np.sum(corect_logprobs) / N

    return loss

In [ ]:
def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [ ]:
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = (4 * 20) + (20 * 3) + 20 + 3
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(objective_func=f, iters=1000, verbose=3)

2022-01-27 23:54:39,334 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.00275
2022-01-27 23:55:08,915 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0027495354316405605, best pos: [-1.98077318e-01 -7.98043169e-01 -5.93924042e-01  3.46552785e-01
  1.02891541e+00  4.78867365e-01 -3.92334277e-01  3.01152763e-01
 -1.18260051e+00  8.93320145e-01 -6.68367589e-01 -2.63540203e+00
  1.62905930e+00 -4.98823018e-01 -4.97724130e-01  6.00572593e-01
  5.31562201e-01  2.76627278e+00 -3.84580222e-01  4.14006362e-01
  2.00943944e-02 -6.95539689e+00  1.24162557e+00  1.11867417e+00
  2.30303891e+00  1.51731045e+00  4.97555584e+00  1.15499545e+00
  1.77568057e+00 -3.86254543e-01  7.69681671e-01 -1.62539823e-01
  3.10468944e+00 -4.43932115e-01  1.20149355e+00  1.16204714e+00
  4.55443399e-01  2.18681285e+00  3.21939786e-01  3.67189729e+00
  2.94243029e+00 -2.

In [ ]:
def predict(X, pos):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    X: numpy.ndarray
        Input Iris dataset
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    # Neural network architecture
    n_inputs = 4
    n_hidden = 20
    n_classes = 3

    # Roll-back the weights and biases
    W1 = pos[0:80].reshape((n_inputs,n_hidden))
    b1 = pos[80:100].reshape((n_hidden,))
    W2 = pos[100:160].reshape((n_hidden,n_classes))
    b2 = pos[160:163].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [ ]:
(predict(X, pos) == y).mean()

1.0

### Сделайте краткий обзор научной работы, посвящённой алгоритму нейронных сетей, 	 не рассматриваемому ранее на курсе. Проведите анализ: чем отличается выбранная архитектура от других? В чём плюсы и минусы данной архитектуры? Какие могут возникнуть трудности при её применении на практике?

Particle swarm optimizer эффективный механизм, исходя из статей, приведенных выше. Для более глубокого изучения была использована дополнительная [статья](https://kpfu.ru/staff_files/F_1407356997/overview.pdf), которая раскрывает суть PSO, но не стала доступной для моего индивидуального понимания.